In [ ]:
import json
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import mplhep as mh

from hist.hist import Hist
from hist.axis import Regular
from hist.axis import IntCategory
from hist.axis import StrCategory

from pathlib import Path

from typing import Optional, Union

from matplotlib.colors import Colormap, ListedColormap
from matplotlib.colors import LogNorm

In [ ]:
from functools import cache
from functools import cached_property

@cache
def get_segment(ring: int, station: int, sector: int, subsector: int) -> int:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L361-L368
    """
    nsub = 3 if ring == 1 and station > 1 else 6
    return subsector + nsub * (sector - 1)


@cache
def get_roll_name(region: int, ring: int, station: int, sector: int, layer: int,
             subsector: int, roll: int
) -> str:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L11-L87
    """
    if region == 0:
        name = f'W{ring:+d}_RB{station}'

        if station <= 2:
            name += 'in' if layer == 1 else 'out'
        else:
            if sector == 4 and station == 4:
                name += ['--', '-', '+', '++'][subsector - 1]
            elif (station == 3) or (station == 4 and sector not in (4, 9, 11)):
                name += '-' if subsector == 1 else '+'
        name += f'_S{sector:0>2d}_'
        name += ['Backward', 'Middle', 'Forward'][roll - 1]
    else:
        segment = get_segment(ring, station, sector, subsector)
        name = f'RE{station * region:+d}_R{ring}_CH{segment:0>2d}_'
        name += ['A', 'B', 'C', 'D', 'E'][roll - 1]
    return name

In [ ]:
def load_data(
    input_path: Path,
    columns: list,
    roll_blacklist_path: Optional[Path] = None,
) -> dict:
    #############################################################
    ##     COLUMNS
    ##     'is_fiducial', 'is_matched', 
    ##     'region', 'ring', 'station', 'sector', 'layer', 'subsector', 'roll', 
    ##     'run', 'cls', 'bx', 'event',
    ##     'tag_pt', 'tag_eta', 'tag_phi', 
    ##     'probe_pt', 'probe_eta', 'probe_phi', 'probe_time', 'probe_dxdz', 'probe_dydz', 
    ##     'dimuon_pt', 'dimuon_mass', 
    ##     'residual_x', 'residual_y', 'pull_x', 'pull_y', 'pull_x_v2', 'pull_y_v2', 
    #############################################################
    data = uproot.open(f"{str(input_path)}:tree").arrays(columns, library='np')
    
    fiducial_mask = data['is_fiducial']
    for key, values in data.items():
        data[key] = data[key][fiducial_mask]

    data['roll_name'] = np.array([
        get_roll_name(
            data['region'][idx], data['ring'][idx], data['station'][idx],
            data['sector'][idx], data['layer'][idx], data['subsector'][idx], data['roll'][idx]
        ) for idx in range(len(data['region']))
    ])

    if roll_blacklist_path is None:
        roll_blacklist = set()
    else:
        with open(roll_blacklist_path) as stream:
            roll_blacklist = set(json.load(stream))
    
    is_blacklist = np.vectorize(lambda item: item in roll_blacklist)
    blacklist_mask = is_blacklist(data['roll_name'])

    for key, values in data.items():
        data[key] = data[key][~blacklist_mask]

    return data

def load_region_data(
    data: dict,
    region: str
):
    if region == "all":
        is_region = np.vectorize(lambda item: type(item) is str)
    elif region == "barrel":
        is_region = np.vectorize(lambda item: item.startswith('W'))
    elif region == "disk123":
        is_region = np.vectorize(lambda item: item.startswith('RE') and ~(item.startswith('RE+4') or item.startswith('RE-4')))
    elif region == "disk4":
        is_region = np.vectorize(lambda item: item.startswith('RE') and (item.startswith('RE+4') or item.startswith('RE-4')))

    region_mask = is_region(data['roll_name'])

    region_data = {}
    for key, values in data.items():
        region_data[key] = data[key][region_mask]

    return region_data

In [ ]:
def plot_cls_hist(
    data: dict,
    region: str = "all",
    ax: Optional[plt.Axes] = None,
    facecolor: str = "#d3f5e4",
    edgecolor: str = "#007700",
    data_legend: str = "2022",
    hatch = None
):
    data = load_region_data(data = data, region = region)
    cls_mask = data['cls'] > 0
    for key, values in data.items():
        data[key] = data[key][cls_mask]    
    
    hist, bins = np.histogram(data['cls'], bins=11, range=(0, 11))
    mh.histplot(
        hist,
        bins=bins - 0.5,
        ax=ax,
        yerr = False,
        histtype="fill",
        facecolor=facecolor,
        edgecolor=edgecolor,
        linewidth=1.6,
        flow=None,
        label=f"{data_legend}      {np.mean(data['cls']) : .3f}\t ",
        alpha=0.5,
        hatch=hatch,
        density=True,
    )
    return ax

In [ ]:
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')

columns = [
    'is_fiducial', 'is_matched', 
    'region', 'ring', 'station', 'sector', 'layer', 'subsector', 'roll', 
    'run', 'cls', 'bx', 'event', 
    'residual_x', 'residual_y',
    'dimuon_mass', 
]

data2022 = load_data(
    input_path = working_dir / 'data' / 'Run2022.root',
    roll_blacklist_path = working_dir / 'blacklist' / 'roll-blacklist.json',
    columns = columns
)

data2023 = load_data(
    input_path = working_dir / 'data' / 'Run2023.root',
    roll_blacklist_path = working_dir / 'blacklist' / 'roll-blacklist.json',
    columns = columns
)

In [ ]:
###########################
## Plotting Cluster Size Distribution
###########################
mh.style.use(mh.styles.CMS)
label = "Work in Progress"
com = 13.6
regions = ['barrel', 'disk123', 'all', 'disk4']
region_labels = ['Barrel', 'Endcap(without Disk4)', 'All', 'Endcap(Disk4)']
facecolors = [("#d3f5e4", "#21bf70"), ("#7CA1FF", "#0714FF"), ("#8EFFF9", "#00AEC9"), ("#FF6666", "#FF3300")]
edgecolors = ["#007700", "#000775", "#005F77", "#CC0000"]

for idx in range(len(regions)):
    fig, ax = plt.subplots(figsize=(16, 10))
    mh.cms.label(ax=ax, data=True, label=label, com=com, year=f"RPC - {region_labels[idx]}", fontsize=30)
    ax.set_xlabel('Cluster Size', fontsize=24)
    ax.set_ylabel('Nomalized', fontsize=24)
    ax.set_xlim(0.5, 10.5)
    ax.set_xticks([x for x in range(1, 11)])
    ax.set_ylim(0, 0.65)
    ax.plot(np.NaN, np.NaN, '-', color="None", label="               Mean")

    ax = plot_cls_hist(
        data = data2022,
        region = regions[idx],
        ax = ax,
        facecolor= facecolors[idx][0],
        edgecolor = edgecolors[idx],
        data_legend = "2022",
        hatch = "///"
    )

    ax = plot_cls_hist(
        data = data2023,
        region = regions[idx],
        ax = ax,
        facecolor = facecolors[idx][1],
        edgecolor = edgecolors[idx],
        data_legend = "2023",
        hatch = None
    )

    ax.legend(fontsize="26", loc='upper right')

    output_dir = working_dir / 'plotting' / 'Hist1D' / 'hit-cls'

    if not output_dir.exists():
        output_dir.mkdir(parents=True)

    fig.savefig(output_dir / f"Run3-{regions[idx]}.png")